Basic chatbot to act as a clinician to diagnose clinical cases

Load required libraries

see the following URL
https://docs.science.ai.cam.ac.uk/hands-on-llms/2_open_ai/



In [20]:
from openai import OpenAI

In [21]:
import os
import dotenv
from rich import print as rprint

Call chatbot OpenAI API

In [22]:
dotenv.load_dotenv()
# load .env

# get API KEY and store it in variable
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# call OpenAI function
client = OpenAI()


Call API

In [ ]:
system_prompt = "You are a clinician and you are diagnosing cases. Given the prompt please give a diagnosis. Please explain all the steps in your reasoning process."
user_query = "A patient has come to you complaining about chest pain. Please give a diagnosis. Please explain all the steps."

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content":system_prompt},
        {"role":"user", "content":user_query}
    ],
    max_tokens=1000
)

print(response.choices[0].message.content)
